In [2]:
# increase the cell width 
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; } </style>"))   

# need to run this every time start this notebook, to add python3.7/site-packages to sys.pat, in order to import ipywidgets, which is used when RobertaTokenizer.from_pretrained('roberta-base') 
# import sys
# sys.path.insert(0, '/xdisk/msurdeanu/fanluo/miniconda3/lib/python3.8/site-packages') 
# sys.path.insert(0, '/xdisk/msurdeanu/fanluo/miniconda3/envs/hotpotqa/lib/python3.7/site-packages')  
# sys.path.insert(0, '/home/u32/fanluo/.ipython') 

### convert hotpotqa to squard format

According to Longformer: use the following input format with special tokens:  “[CLS] [q] question [/q] [p] sent1,1 [s] sent1,2 [s] ... [p] sent2,1 [s] sent2,2 [s] ...” 
where [s] and [p] are special tokens representing sentences and paragraphs. The special tokens were added to the RoBERTa vocabulary and randomly initialized before task finetuning.

In [3]:
# helper functions to convert hotpotqa to squard format modified from  https://github.com/chiayewken/bert-qa/blob/master/run_hotpot.py

import tqdm 
from datetime import datetime 
import pytz 
timeZ_Az = pytz.timezone('US/Mountain') 
import transformers 

# QUESTION_START = '[question]'
# QUESTION_END = '[/question]' 
# TITLE_START = '<t>'  # indicating the start of the title of a paragraph (also used for loss over paragraphs)
# TITLE_END = '</t>'   # indicating the end of the title of a paragraph
# SENT_MARKER_END = '[/sent]'  # indicating the end of the title of a sentence (used for loss over sentences)
# PAR = '[/par]'  # used for indicating end of the regular context and beginning of `yes/no/null` answers
EXTRA_ANSWERS = " yes no null </s> "

 
def create_example_dict(context, answer, id, question, is_sup_fact, is_supporting_para):
    return {
        "context": context,
        "qas": [                        # each context corresponds to only one qa in hotpotqa
            {
                "answer": answer,
                "id": id,
                "question": question,
                "is_sup_fact": is_sup_fact,
                "is_supporting_para": is_supporting_para
            }
        ],
    }

def create_para_dict(example_dicts):
    if type(example_dicts) == dict:
        example_dicts = [example_dicts]   # each paragraph corresponds to only one [context, qas] in hotpotqa
    return {"paragraphs": example_dicts}   


In [4]:
import re
import string

def convert_hotpot_to_squad_format(json_dict, gold_paras_only=False):
    
    """function to convert hotpotqa to squard format.


    Note: A context corresponds to several qas in SQuard. In hotpotqa, one question corresponds to several paragraphs as context. 
          "paragraphs" means different: each paragraph in SQuard contains a context and a list of qas; while 10 paragraphs in hotpotqa concatenated into a context for one question.

    Args:
        json_dict: The original data load from hotpotqa file.
        gold_paras_only: when is true, only use the 2 paragraphs that contain the gold supporting facts; if false, use all the 10 paragraphs
 

    Returns:
        new_dict: The converted dict of hotpotqa dataset, use it as a dict would load from SQuAD json file
                  usage: input_data = new_dict["data"]   https://github.com/google-research/bert/blob/eedf5716ce1268e56f0a50264a88cafad334ac61/run_squad.py#L230

    """
 
    new_dict = {"data": []} 
    for example in json_dict: 

        support_para = set(
            para_title for para_title, _ in example["supporting_facts"]
        )
        sp_set = set(list(map(tuple, example['supporting_facts'])))
        
        raw_contexts = example["context"]
        if gold_paras_only: 
            raw_contexts = [lst for lst in raw_contexts if lst[0] in support_para]
            
        is_supporting_para = []  # a boolean list with 10 True/False elements, one for each paragraph
        is_sup_fact = []         # a boolean list with True/False elements, one for each context sentence
        for para_title, para_lines in raw_contexts:
            is_supporting_para.append(para_title in support_para)   
            for sent_id, sent in enumerate(para_lines):
                is_sup_fact.append( (para_title, sent_id) in sp_set )    
        
        for lst in raw_contexts:
            lst[0] = _normalize_text(lst[0])
            lst[1] = [_normalize_text(sent) for sent in lst[1]]
        
        contexts = [lst[0]  + ' '  + ' '.join(lst[1]) for lst in raw_contexts]    
        # extra space is fine, which would be ignored latter. most sentences has already have heading space, there are several no heading space; call the _normalize_text() which is same as the one used during evaluation
        
#         context = " </s> ".join(contexts)
#         print(context)
        
#         exit(0)

        
        answer = _normalize_text(example["answer"]) 
#         print("answer: ", answer)
        if(len(answer) > 0):   # answer can be '' after normalize
            new_dict["data"].append(
                create_para_dict(
                    create_example_dict(
                        context=contexts,
                        answer=answer,
                        id = example["_id"],
                        question=_normalize_text(example["question"]),
                        is_sup_fact = is_sup_fact,
                        is_supporting_para = is_supporting_para 
                    )
                )
            ) 

    return new_dict

def _normalize_text(s):

    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

In [5]:
# debug: check whether convert_hotpot_to_squad_format() works
import os
os.chdir('/xdisk/msurdeanu/fanluo/hotpotQA/Data')
# !cat /xdisk/msurdeanu/fanluo/hotpotQA/Data/small.json | ../../helper/jq-linux64 -c '.[30:40]' > small_3.json
# !cat /xdisk/msurdeanu/fanluo/hotpotQA/Data/hotpot_train_v1.1.json | ../../helper/jq-linux64 -c '.[76200:76280]' > small.json
#!cat /xdisk/msurdeanu/fanluo/hotpotQA/Data/hotpot_train_v1.1.json | ../../helper/jq-linux64 -c '.[37:50]' > small_dev.json
# !cat /xdisk/msurdeanu/fanluo/hotpotQA/Data/hotpot_train_v1.1.json | ../../helper/jq-linux64 -c '.[31:50]' > sample.json

import json
with open("small.json", "r", encoding='utf-8') as f:  
    json_dict = convert_hotpot_to_squad_format(json.load(f))['data']
    print(json.dumps(json_dict[3], indent=2))

{
  "paragraphs": [
    {
      "context": [
        "dr seuss how grinch stole christmas dr seuss how grinch stole christmas is video game based on dr seuss book with same name but mostly based on film game was released on november 8 2007",
        "lorax film lorax also known as dr seuss lorax is 2012 american 3d computeranimated musical fantasy\u2013comedy film produced by illumination entertainment and based on dr seusss childrens book of same name film was released by universal pictures on march 2 2012 on 108th birthday of dr seuss second film adaptation of book following 1972 animated television special film builds on book by expanding story of ted previously unnamed boy who visits onceler cast includes danny devito as lorax ed helms as onceler and zac efron as ted new characters introduced in film are audrey voiced by taylor swift aloysius ohare rob riggle mrs wiggins teds mother jenny slate and grammy norma betty white",
        "horton hears who tv special horton hears who is 

### longfomer's fine-tuning


- For answer span extraction we use BERT’s QA model with addition of a question type (yes/no/span) classification head over the first special token ([CLS]).

- For evidence extraction we apply 2 layer feedforward networks on top of the representations corresponding to sentence and paragraph tokens to get the corresponding evidence prediction scores and use binary cross entropy loss to train the model.

- We combine span, question classification, sentence, and paragraphs losses and train the model in a multitask way using linear combination of losses.


In [6]:
### Section2: This is modified from longfomer's fine-tuning with triviaqa.py from https://github.com/allenai/longformer/blob/master/scripts/triviaqa.py

# !python -m pip install --upgrade git+http://github.com/ibeltagy/transformers.git@longformer_encoder_decoder#egg=transformers
# !conda install cudatoolkit=10.0 --yes
# !python -m pip install git+https://github.com/allenai/longformer.git
####requirements.txt:torch>=1.2.0, transformers>=3.0.2, tensorboardX, pytorch-lightning==0.6.0, test-tube==0.7.5
# !conda install -c conda-forge regex --force-reinstall --yes
# !conda install pytorch-lightning -c conda-forge
#!python -m pip install jdc 
# !pip install test-tube 
#!python -m pip install ipywidgets 
#!conda update --force conda --yes  
# !jupyter nbextension enable --py widgetsnbextension 
# !conda install jupyter --yes


import os
import json
import string
import random
import numpy as np
import torch
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.data import DataLoader, Dataset 

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.overrides.data_parallel import LightningDistributedDataParallel
from pytorch_lightning.logging import TestTubeLogger    # sometimes pytorch_lightning.loggers works instead

from longformer.longformer import Longformer, LongformerConfig
from longformer.sliding_chunks import pad_to_window_size
from transformers import RobertaTokenizer
import jdc
from more_itertools import locate
from collections import Counter
from collections import defaultdict
from transformers import DistilBertTokenizer, DistilBertModel


In [7]:
print(pl.__file__)

/home/u32/fanluo/.local/lib/python3.6/site-packages/pytorch_lightning/__init__.py


#### class hotpotqaDataset

##### \_\_init\_\_, \_\_getitem\_\_ and \_\_len\_\_ 

In [8]:
class hotpotqaDataset(Dataset):
    """
    Largely based on
    https://github.com/allenai/allennlp/blob/master/allennlp/data/dataset_readers/reading_comprehension/triviaqa.py
    and
    https://github.com/huggingface/transformers/blob/master/examples/run_squad.py
    """
    
    
    def __init__(self, file_path, tokenizer, max_seq_len, max_doc_len, doc_stride,
                 max_num_answers, ignore_seq_with_no_answers, max_question_len):
        assert os.path.isfile(file_path)
        self.file_path = file_path
        if("reduced_context" not in self.file_path):
            with open(self.file_path, "r", encoding='utf-8') as f:
                print(f'reading file: {self.file_path}')
                self.data_json = convert_hotpot_to_squad_format(json.load(f))['data']
                
        else:
            with open(self.file_path, "r", encoding='utf-8') as f:
                print(f'reading file: {self.file_path}')
                self.data_json = json.load(f)['data']            
                print(self.data_json[0])
            
        self.tokenizer = tokenizer
        self.max_seq_len = max_seq_len
        self.max_doc_len = max_doc_len
        self.doc_stride = doc_stride
        self.max_num_answers = max_num_answers
        self.ignore_seq_with_no_answers = ignore_seq_with_no_answers
        self.max_question_len = max_question_len


#         print(tokenizer.all_special_tokens) 
    
        # A mapping from qid to an int, which can be synched across gpus using `torch.distributed`
        if 'train' not in self.file_path:  # only for the evaluation set 
            self.val_qid_string_to_int_map =                  {
                    entry["paragraphs"][0]['qas'][0]['id']: index
                    for index, entry in enumerate(self.data_json)
                }
        else:
            self.val_qid_string_to_int_map = None
            
            
    def __len__(self):
        return len(self.data_json)

    def __getitem__(self, idx):
        entry = self.data_json[idx]
        tensors_list = self.one_example_to_tensors(entry, idx)
        if(len(tensors_list) != 1):
            print("tensors_list: ", tensors_list)
        assert len(tensors_list) == 1
        return tensors_list[0]

##### one_example_to_tensors

In [9]:
    %%add_to hotpotqaDataset
    def one_example_to_tensors(self, example, idx):
        def is_whitespace(c):
            if c == " " or c == "\t" or c == "\r" or c == "\n" or ord(c) == 0x202F:
                return True
            return False
        
        def map_answer_positions(answer_start, answer_end, context_ids, context_offset):
            # form ids' character offset to tokens' positon
            context_ids_str = " ".join(map(str, context_ids)) 
            start_position = context_ids_str[:answer_start].count(" ") + context_offset
            end_position = context_ids_str[:answer_end-1].count(" ") + context_offset
  
            return (start_position, end_position)
        
        tensors_list = []
        for paragraph in example["paragraphs"]:  # example["paragraphs"] only contains one paragraph in hotpotqa
            context = self.tokenizer.sep_token + ' ' + (' ' + self.tokenizer.sep_token + ' ').join(paragraph["context"] )
            context = EXTRA_ANSWERS + context
            context = ' '.join(context.split()) # white space fix
#             print("self.tokenizer.sep_token: ", self.tokenizer.sep_token)
#             print("self.tokenizer.sep_token == '</s>': ", self.tokenizer.sep_token == '</s>')
            
#             context = '[SEP]' + ' ' + (' [SEP] ').join(paragraph["context"] )
#             context = EXTRA_ANSWERS + context
#             context = ' '.join(context.split())
            
#             doc_tokens = []
#             char_to_word_offset = []
#             prev_is_whitespace = True
#             for c in context:
#                 if is_whitespace(c):
#                     prev_is_whitespace = True
#                 else:
#                     if prev_is_whitespace:
#                         doc_tokens.append(c) # add a new token
#                     else:
#                         doc_tokens[-1] += c  # append the character to the last token
#                     prev_is_whitespace = False
#                 char_to_word_offset.append(len(doc_tokens) - 1)
            
#             print("len(char_to_word_offset): ", len(char_to_word_offset))
#             print("char_to_word_offset: ", char_to_word_offset)
            for qa in paragraph["qas"]:
                question_text = qa["question"]
#                 print("question text: ", question_text)  
                sp_sent = qa["is_sup_fact"]
                sp_para = qa["is_supporting_para"]
                start_position = None
                end_position = None
                orig_answer_text = None 

#                     print("len(sp_sent):", len(sp_sent))
#                     print("sp_sent", sp_sent) 
#                     print("doc_tokens", doc_tokens)
 
                # keep all answers in the document, not just the first matched answer. It also added the list of textual answers to make evaluation easy.
                
                   
                # ===== Given an example, convert it into tensors  =============
                 
#                 query_tokens = self.tokenizer.tokenize(question_text)
                question_text = " ".join(question_text.split()[:self.max_question_len])
#                 tok_to_orig_index = []
#                 orig_to_tok_index = []
#                 all_doc_tokens = []
                
#                 # each original token in the context is tokenized to multiple sub_tokens
#                 for (i, token) in enumerate(doc_tokens):
#                     orig_to_tok_index.append(len(all_doc_tokens))
#                     # hack: the line below should have been `self.tokenizer.tokenize(token')`
#                     # but roberta tokenizer uses a different subword if the token is the beginning of the string
#                     # or in the middle. So for all tokens other than the first, simulate that it is not the first
#                     # token by prepending a period before tokenizing, then dropping the period afterwards
#                     sub_tokens = self.tokenizer.tokenize(f'. {token}')[1:] if i > 0 else self.tokenizer.tokenize(token)
#                     for sub_token in sub_tokens:
#                         tok_to_orig_index.append(i)
#                         all_doc_tokens.append(sub_token)
                
#                 # all sub tokens, truncate up to limit
#                 all_doc_tokens = all_doc_tokens[:self.max_doc_len-8] 

#                 # The -8 accounts for CLS, QUESTION_START, QUESTION_END， [/par]， yes， no， null， </s>   
#                 max_tokens_per_doc_slice = self.max_seq_len - len(query_tokens) - 8
#                 if(max_tokens_per_doc_slice <= 0):
#                     print("(max_tokens_per_doc_slice <= 0)")
#                 assert max_tokens_per_doc_slice > 0
#                 if self.doc_stride < 0:                           # default
#                     # negative doc_stride indicates no sliding window, but using first slice
#                     self.doc_stride = -100 * len(all_doc_tokens)  # large -negtive value for the next loop to execute once
                
                # inputs to the model
                input_ids_list = []
                input_mask_list = []
#                 segment_ids_list = []
                start_positions_list = []
                end_positions_list = []
                q_type_list = []
                sp_sent_list =  [1 if ss else 0 for ss in sp_sent]
                sp_para_list = [1 if sp else 0 for sp in sp_para]
                
                inputs = self.tokenizer(question_text, context, return_tensors='pt', truncation=True, max_length=512) 
                
                input_ids = inputs['input_ids'][0].tolist()
                input_mask = [1] * len(input_ids)
#                 print("size of input_ids in one_example", str(len(input_ids)))
#                 print("id: ", qa['id'], "question_text: ", question_text, "context: ", context)
                    
                sep_token_id = self.tokenizer.convert_tokens_to_ids(self.tokenizer.sep_token)
                context_offset = input_ids.index(sep_token_id)+1
                context_ids = input_ids[context_offset:]
# #                 print("before for")
#                 for slice_start in range(0, len(all_doc_tokens), max_tokens_per_doc_slice - self.doc_stride):    # execute once by default
#                     slice_end = min(slice_start + max_tokens_per_doc_slice, len(all_doc_tokens))

#                     doc_slice_tokens = all_doc_tokens[slice_start:slice_end]
#                     tokens = [self.tokenizer.cls_token] + [QUESTION_START] + query_tokens + [QUESTION_END] + doc_slice_tokens + [PAR] + self.tokenizer.tokenize("yes") + self.tokenizer.tokenize("no") + self.tokenizer.tokenize("null") +  [self.tokenizer.eos_token]   
#                     segment_ids = [0] * (len(query_tokens) + 3) + [1] * (len(doc_slice_tokens) + 5) 
# #                     if(len(segment_ids) != len(tokens)):
# #                         print("len(segment_ids): ", len(segment_ids))
# #                         print("len(tokens): ", len(tokens))
#                     assert len(segment_ids) == len(tokens)

#                     input_ids = self.tokenizer.convert_tokens_to_ids(tokens)   
#                     input_mask = [1] * len(input_ids)

#                     doc_offset = len(query_tokens) + 3 - slice_start  # where context starts
                    
                    # ===== answer positions tensors  ============

                start_positions = []
                end_positions = []

                answer = qa["answer"] 
#                 print("answer: ", answer)
                if answer == 'yes':
                    q_type = 1
                    start_positions.append(context_offset)   
                    end_positions.append(context_offset) 
                elif answer == 'no':
                    q_type = 2
                    start_positions.append(context_offset+1)   
                    end_positions.append(context_offset+1)  
                else:
                    # keep all the occurences of answer in the context 
#                         for m in re.finditer("\s?".join(answer.split()), context):   # "\s?".join(answer.split()) in order to match even with extra space in answer or context

                    answer_ids = self.tokenizer(_normalize_text(answer), return_tensors='pt',max_length=512,truncation=True)['input_ids'][0].tolist()[1:-1]


                    for m in re.finditer(" ".join(map(str, answer_ids)), " ".join(map(str, context_ids)), re.IGNORECASE):
                        answer_start, answer_end = m.span() 
                        start_position, end_position = map_answer_positions(answer_start, answer_end, context_ids, context_offset)
#                             if(start_position != -1):
                        start_positions.append(start_position)   
                        end_positions.append(end_position)

                    if(len(start_positions) > 0): 
                        q_type = 0
                    else: # answer not found in context
                        q_type = 3 
                        start_positions.append(context_offset+2)   
                        end_positions.append(context_offset+2)  
                            

                # answers from start_positions and end_positions if > self.max_num_answers
                start_positions = start_positions[:self.max_num_answers]
                end_positions = end_positions[:self.max_num_answers]

                # -1 padding up to self.max_num_answers
                padding_len = self.max_num_answers - len(start_positions)
                start_positions.extend([-1] * padding_len)
                end_positions.extend([-1] * padding_len)

                # replace duplicate start/end positions with `-1` because duplicates can result into -ve loss values
                found_start_positions = set()
                found_end_positions = set()
                for i, (start_position, end_position) in enumerate(zip(start_positions, end_positions)):

                    if start_position in found_start_positions:
                        start_positions[i] = -1
                    if end_position in found_end_positions:
                        end_positions[i] = -1
                    found_start_positions.add(start_position)
                    found_end_positions.add(end_position)

#                         # for debug
#                         if(start_position != -1):
#                             answer_token_ids = input_ids[start_position: end_position+1]
#                             answer_tokens = self.tokenizer.convert_ids_to_tokens(answer_token_ids)
#                             answer_text = self.tokenizer.convert_tokens_to_string(answer_tokens)
#                             print("answer_text: ", answer_text)

#                     if self.doc_stride >= 0:  # no need to pad if document is not strided
                    # Zero-pad up to the sequence length.
#                 padding_len = self.max_seq_len - len(input_ids)
# #                 print("size of input_ids before padding", str(len(input_ids)))
#                 input_ids.extend([self.tokenizer.pad_token_id] * padding_len)
# #                 print("size of input_ids after padding", str(len(input_ids)))
#                 input_mask = [1] * len(input_ids)
#                 segment_ids = [0]  * len(input_ids)

#                 assert len(input_ids) == self.max_seq_len
#                 assert len(input_mask) == self.max_seq_len
#                 assert len(segment_ids) == self.max_seq_len  

                input_ids_list.append(input_ids)
#                 print("input_ids_list: ", input_ids_list)
                input_mask_list.append(input_mask)
#                 segment_ids_list.append(segment_ids)
                start_positions_list.append(start_positions)
                end_positions_list.append(end_positions)
                q_type_list.append(q_type)
                    
                tensors_list.append((torch.tensor(input_ids_list), torch.tensor(input_mask_list), 
#                                      torch.tensor(segment_ids_list),
                                     torch.tensor(start_positions_list), torch.tensor(end_positions_list), torch.tensor(q_type_list),
                                     qa['id'], answer))     
#             print("tensors_list: ", tensors_list)
        return tensors_list


##### collate_one_doc_and_lists

In [10]:
    %%add_to hotpotqaDataset
    @staticmethod
    def collate_one_doc_and_lists(batch):
        num_metadata_fields = 2  # qid and answer  
        fields = [x for x in zip(*batch)]
        stacked_fields = [torch.stack(field) for field in fields[:-num_metadata_fields]]  # don't stack metadata fields
        stacked_fields.extend(fields[-num_metadata_fields:])  # add them as lists not torch tensors

        # always use batch_size=1 where each batch is one document
        # will use grad_accum to increase effective batch size
        assert len(batch) == 1
        fields_with_batch_size_one = [f[0] for f in stacked_fields]
        return fields_with_batch_size_one


##### class info

In [11]:
dir(hotpotqaDataset)

['__add__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'collate_one_doc_and_lists',
 'one_example_to_tensors']

In [12]:
from inspect import getmembers
getmembers(hotpotqaDataset)

[('__add__', <function torch.utils.data.dataset.Dataset.__add__(self, other)>),
 ('__class__', type),
 ('__delattr__', <slot wrapper '__delattr__' of 'object' objects>),
 ('__dict__',
  mappingproxy({'__module__': '__main__',
                '__doc__': '\n    Largely based on\n    https://github.com/allenai/allennlp/blob/master/allennlp/data/dataset_readers/reading_comprehension/triviaqa.py\n    and\n    https://github.com/huggingface/transformers/blob/master/examples/run_squad.py\n    ',
                '__init__': <function __main__.hotpotqaDataset.__init__(self, file_path, tokenizer, max_seq_len, max_doc_len, doc_stride, max_num_answers, ignore_seq_with_no_answers, max_question_len)>,
                '__len__': <function __main__.hotpotqaDataset.__len__(self)>,
                '__getitem__': <function __main__.hotpotqaDataset.__getitem__(self, idx)>,
                'one_example_to_tensors': <function __main__.one_example_to_tensors(self, example, idx)>,
                'collate_one

In [13]:
from inspect import isfunction
functions_list = [o for o in getmembers(hotpotqaDataset) if isfunction(o[1])]
functions_list

[('__add__', <function torch.utils.data.dataset.Dataset.__add__(self, other)>),
 ('__getitem__', <function __main__.hotpotqaDataset.__getitem__(self, idx)>),
 ('__init__',
  <function __main__.hotpotqaDataset.__init__(self, file_path, tokenizer, max_seq_len, max_doc_len, doc_stride, max_num_answers, ignore_seq_with_no_answers, max_question_len)>),
 ('__len__', <function __main__.hotpotqaDataset.__len__(self)>),
 ('collate_one_doc_and_lists',
  <function __main__.collate_one_doc_and_lists(batch)>),
 ('one_example_to_tensors',
  <function __main__.one_example_to_tensors(self, example, idx)>)]

In [14]:
import inspect
inspect.getmro(hotpotqaDataset)  # a hierarchy of classes 

(__main__.hotpotqaDataset, torch.utils.data.dataset.Dataset, object)

In [15]:
inspect.getfullargspec(hotpotqaDataset.one_example_to_tensors)

FullArgSpec(args=['self', 'example', 'idx'], varargs=None, varkw=None, defaults=None, kwonlyargs=[], kwonlydefaults=None, annotations={})

In [16]:
help(hotpotqaDataset)

Help on class hotpotqaDataset in module __main__:

class hotpotqaDataset(torch.utils.data.dataset.Dataset)
 |  Largely based on
 |  https://github.com/allenai/allennlp/blob/master/allennlp/data/dataset_readers/reading_comprehension/triviaqa.py
 |  and
 |  https://github.com/huggingface/transformers/blob/master/examples/run_squad.py
 |  
 |  Method resolution order:
 |      hotpotqaDataset
 |      torch.utils.data.dataset.Dataset
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __getitem__(self, idx)
 |  
 |  __init__(self, file_path, tokenizer, max_seq_len, max_doc_len, doc_stride, max_num_answers, ignore_seq_with_no_answers, max_question_len)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __len__(self)
 |  
 |  one_example_to_tensors(self, example, idx)
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  collate_one_doc_and_lists(batch)
 |  
 |  -----------------------

#### class hotpotqa

##### \_\_init\_\_,  forward, dataloaders

In [17]:
class hotpotqa(pl.LightningModule):
    def __init__(self, args):
        super(hotpotqa, self).__init__()
        self.args = args
        self.hparams = args
 
        self.tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
#         self.tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
#         num_new_tokens = self.tokenizer.add_special_tokens({"additional_special_tokens": [TITLE_START, TITLE_END, SENT_MARKER_END, QUESTION_START , QUESTION_END, PAR]})
#         print(self.tokenizer.all_special_tokens)
        self.tokenizer.model_max_length = self.args.max_seq_len
        self.model = self.load_model()
#         self.model.resize_token_embeddings(len(self.tokenizer))
        
        self.num_labels = 2  # start/end
        self.qa_outputs = torch.nn.Linear(self.model.config.hidden_size, self.num_labels)
        self.linear_type = torch.nn.Linear(self.model.config.hidden_size, 4)   #  question type (yes/no/span/null) classification 
        
#         self.fnn_sp_sent = torch.nn.Sequential(
#           torch.nn.Linear(self.model.config.hidden_size, self.model.config.hidden_size), 
#           torch.nn.GELU(),
#           torch.nn.Linear(self.model.config.hidden_size, 1),      # score for 'yes', while 0 for 'no'
#         )
        
#         self.fnn_sp_para = torch.nn.Sequential(
#           torch.nn.Linear(self.model.config.hidden_size, self.model.config.hidden_size), 
#           torch.nn.GELU(),
#           torch.nn.Linear(self.model.config.hidden_size, 1),      # score for 'yes', while 0 for 'no'
#         )
         
        
        self.train_dataloader_object = self.val_dataloader_object  = None #= self.test_dataloader_object = None
        
 
    def load_model(self):
        
#         config = LongformerConfig.from_pretrained(self.args.model_path) 
#         config.attention_mode = self.args.attention_mode
#         model = Longformer.from_pretrained(self.args.model_path, config=config)

        model = DistilBertModel.from_pretrained('distilbert-base-uncased')

#         tokenizer = tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased') #AutoTokenizer.from_pretrained("/xdisk/msurdeanu/fanluo/hotpotQA/tiny-longformer-random")

#         model = TFAutoModel.from_pretrained("lysandre/tiny-longformer-random")

#         for layer in model.encoder.layer:
#             layer.attention.self.attention_mode = self.args.attention_mode
#             self.args.attention_window = layer.attention.self.attention_window

        print("Loaded model with config:")
        print(model.config)

        for p in model.parameters():
            p.requires_grad_(True)
        model.train()
        return model

# %%add_to hotpotqa    # does not seems to work for the @pl.data_loader decorator, missing which causes error "validation_step() takes 3 positional arguments but 4 were given"    
    ###################################################### dataloaders ########################################################### 
    
    @pl.data_loader
    def train_dataloader(self):
        if self.train_dataloader_object is not None:
            return self.train_dataloader_object
        dataset = hotpotqaDataset(file_path=self.args.train_dataset, tokenizer=self.tokenizer,
                                  max_seq_len=self.args.max_seq_len, max_doc_len=self.args.max_doc_len,
                                  doc_stride=self.args.doc_stride,
                                  max_num_answers=self.args.max_num_answers,
                                  max_question_len=self.args.max_question_len,
                                  ignore_seq_with_no_answers=self.args.ignore_seq_with_no_answers)
        
#         dist_sampler = torch.utils.data.distributed.DistributedSampler(dataset)
        dl = DataLoader(dataset, batch_size=1, shuffle=False,   # set shuffle=False, otherwise it will sample a different subset of data every epoch with train_percent_check
                        num_workers=self.args.num_workers,  
                        collate_fn=hotpotqaDataset.collate_one_doc_and_lists)

        self.train_dataloader_object = dl  
        return self.train_dataloader_object
    
 

    @pl.data_loader
    def val_dataloader(self):
        if self.val_dataloader_object is not None:
            return self.val_dataloader_object
        dataset = hotpotqaDataset(file_path=self.args.dev_dataset, tokenizer=self.tokenizer,
                                  max_seq_len=self.args.max_seq_len, max_doc_len=self.args.max_doc_len,
                                  doc_stride=self.args.doc_stride,
                                  max_num_answers=self.args.max_num_answers,
                                  max_question_len=self.args.max_question_len,
                                  ignore_seq_with_no_answers=False)  # evaluation data should keep all examples 

        
        
#         dist_sampler = torch.utils.data.distributed.DistributedSampler(dataset)
        dl = DataLoader(dataset, batch_size=1, shuffle=False,
                        num_workers=self.args.num_workers, 
                        collate_fn=hotpotqaDataset.collate_one_doc_and_lists)
        self.val_dataloader_object = dl
        return self.val_dataloader_object

#     @pl.data_loader
#     def test_dataloader(self):
#         if self.test_dataloader_object is not None:
#             return self.test_dataloader_object
#         dataset = hotpotqaDataset(file_path=self.args.dev_dataset, tokenizer=self.tokenizer,
#                                   max_seq_len=self.args.max_seq_len, max_doc_len=self.args.max_doc_len,
#                                   doc_stride=self.args.doc_stride,
#                                   max_num_answers=self.args.max_num_answers,
#                                   max_question_len=self.args.max_question_len,
#                                   ignore_seq_with_no_answers=False)  # evaluation data should keep all examples

# #         dist_sampler = torch.utils.data.distributed.DistributedSampler(dataset)
#         dl = DataLoader(dataset, batch_size=1, shuffle=False,
#                         num_workers=self.args.num_workers, 
#                         collate_fn=hotpotqaDataset.collate_one_doc_and_lists)
#         self.test_dataloader_object = dl
#         return self.test_dataloader_object

    #%%add_to hotpotqa  
    def forward(self, input_ids, start_positions, end_positions, q_type):
 
        if(input_ids.size(0) > 1):
            assert("multi rows per document")
        # Each batch is one document, and each row of the batch is a chunck of the document.    ????
        # Make sure all rows have the same question length.
        
#         # local attention everywhere
#         attention_mask = torch.ones(input_ids.shape, dtype=torch.long, device=input_ids.device)
        
#         # global attention for the cls and all question tokens
#         question_end_index = self._get_special_index(input_ids, [QUESTION_END])
# #         if(question_end_index.size(0) == 1):
# #             attention_mask[:,:question_end_index.item()] = 2  
# #         else:
#         attention_mask[:,:question_end_index[0].item()+1] = 2  # from <cls> until </q>
# #             print("more than 1 <q> in: ", self.tokenizer.convert_ids_to_tokens(input_ids[0].tolist()) )
        
#         # global attention for the sentence and paragraph special tokens  
#         sent_indexes = self._get_special_index(input_ids, [SENT_MARKER_END])
#         attention_mask[:, sent_indexes] = 2
        
#         para_indexes = self._get_special_index(input_ids, [TITLE_START])
#         attention_mask[:, para_indexes] = 2       
         

#         # sliding_chunks implemenation of selfattention requires that seqlen is multiple of window size
#         input_ids, attention_mask = pad_to_window_size(
#             input_ids, attention_mask, self.args.attention_window, self.tokenizer.pad_token_id) 
        
#         print("tokens: ", self.tokenizer.convert_ids_to_tokens(input_ids[0].tolist()))
#         print("size of input_ids", str(input_ids.size()))
        if(input_ids.shape[0] == 0):
            print("input_ids: ", input_ids)
        sequence_output = self.model(input_ids)[0]
#         print("size of sequence_output: " + str(sequence_output.size()))
        print("sequence_output ", sequence_output)
#         (
#                 input_ids,
#                 attention_mask=attention_mask)[0]
#         print("size of sequence_output: " + str(sequence_output.size()))

        # The pretrained hotpotqa model wasn't trained with padding, so remove padding tokens
        # before computing loss and decoding.
        padding_len = input_ids[0].eq(self.tokenizer.pad_token_id).sum()
        if padding_len > 0:
            sequence_output = sequence_output[:, :-padding_len]
#         print("size of sequence_output after removing padding: " + str(sequence_output.size()))
              
        
        ###################################### layers on top of sequence_output ##################################
        

        ### 1. answer start and end positions classification ###   
        logits = self.qa_outputs(sequence_output) 
        start_logits, end_logits = logits.split(1, dim=-1)  
    
        start_logits = start_logits.squeeze(-1) 
        end_logits = end_logits.squeeze(-1)
#         print("start_logits in forward: ", start_logits)
#         print("end_logits in forward: ", end_logits)
#         ### 2. type classification, similar as class LongformerClassificationHead(nn.Module) https://huggingface.co/transformers/_modules/transformers/modeling_longformer.html#LongformerForSequenceClassification.forward ### 
        type_logits = self.linear_type(sequence_output[:,0]) 
        
#         ### 3. supporting paragraph classification ###  
#         sp_para_output = sequence_output[:,para_indexes,:]  
#         sp_para_output_t = self.fnn_sp_para(sp_para_output) 

#          # linear_sp_sent generates a single score for each sentence, instead of 2 scores for yes and no.   
#         # Argument the score with additional score=0. The same way did in the HOTPOTqa paper
#         sp_para_output_aux = torch.zeros(sp_para_output_t.shape, dtype=torch.float, device=sp_para_output_t.device) 
#         predict_support_para = torch.cat([sp_para_output_aux, sp_para_output_t], dim=-1).contiguous() 
 
#         ### 4. supporting fact classification ###     
#         # the first sentence in a paragraph is leading by <p>, other sentences are leading by <s>
 
#         sp_sent_output = sequence_output[:,sent_indexes,:]  
#         sp_sent_output_t = self.fnn_sp_sent(sp_sent_output)     
#         sp_sent_output_aux = torch.zeros(sp_sent_output_t.shape, dtype=torch.float, device=sp_sent_output_t.device) 
#         predict_support_sent = torch.cat([sp_sent_output_aux, sp_sent_output_t], dim=-1).contiguous() 
        
 
        answer_loss, type_loss = self.loss_computation(start_positions, end_positions, start_logits, end_logits, q_type, type_logits) 
#         explainer = shap.GradientExplainer( (logits, sequence_output), self.qa_outputs(sequence_output))
#         print(self.explainer)

        
        return answer_loss, type_loss, start_logits, end_logits, type_logits 
    
    def loss_computation(self, start_positions, end_positions, start_logits, end_logits, q_type, type_logits):
        if start_positions is not None and end_positions is not None:
            # If we are on multi-GPU, split add a dimension
            if len(start_positions.size()) > 1:
                start_positions = start_positions.squeeze(-1)
            if len(end_positions.size()) > 1:
                end_positions = end_positions.squeeze(-1)

#             if not self.args.regular_softmax_loss:
#                 # loss function suggested in section 2.2 here https://arxiv.org/pdf/1710.10723.pdf
#                 # NOTE: this returns sum of losses, not mean, so loss won't be normalized across different batch sizes
#                 # but batch size is always 1, so this is not a problem
            start_loss = self.or_softmax_cross_entropy_loss_one_doc(start_logits, start_positions, ignore_index=-1)
            end_loss = self.or_softmax_cross_entropy_loss_one_doc(end_logits, end_positions, ignore_index=-1)
#             else: 
#             start_positions = start_positions[:, 0:1]   # only use the top1 start_position considering only one appearance of the answer string
#             end_positions = end_positions[:, 0:1]
#             start_loss = crossentropy(start_logits, start_positions[:, 0])
#             end_loss = crossentropy(end_logits, end_positions[:, 0])
            crossentropy = torch.nn.CrossEntropyLoss()
            type_loss = crossentropy(type_logits, q_type)  
 
 
            answer_loss = (start_loss + end_loss) / 2 
        return answer_loss , type_loss


#     %%add_to hotpotqa    
    def _get_special_index(self, input_ids, special_tokens):
        assert(input_ids.size(0)==1) 
        mask = input_ids != input_ids # initilaize 
        for special_token in special_tokens:
            mask = torch.logical_or(mask, input_ids.eq(self.tokenizer.convert_tokens_to_ids(special_token))) 
 
        token_indices = torch.nonzero(mask)    
         
 
        return token_indices[:,1]    

    def or_softmax_cross_entropy_loss_one_doc(self, logits, target, ignore_index=-1, dim=-1):
        """loss function suggested in section 2.2 here https://arxiv.org/pdf/1710.10723.pdf"""
        assert logits.ndim == 2
        assert target.ndim == 2
        assert logits.size(0) == target.size(0) 
        
        # with regular CrossEntropyLoss, the numerator is only one of the logits specified by the target, considing only one correct target 
        # here, the numerator is the sum of a few potential targets, where some of them is the correct answer, considing more correct targets

        # target are indexes of tokens, padded with ignore_index=-1
        # logits are scores (one for each label) for each token
 
        # compute a target mask
        target_mask = target == ignore_index
        # replaces ignore_index with 0, so `gather` will select logit at index 0 for the masked targets
        masked_target = target * (1 - target_mask.long())                 # replace all -1 in target with 0， tensor([[447,   0,   0,   0, ...]])
    
        # gather logits
        gathered_logits = logits.gather(dim=dim, index=masked_target)     # tensor([[0.4382, 0.2340, 0.2340, 0.2340 ... ]]), padding logits are all replaced by logits[0] 
 
        # Apply the mask to gathered_logits. Use a mask of -inf because exp(-inf) = 0
        gathered_logits[target_mask] = float('-inf')                      # padding logits are all replaced by -inf
 
        # each batch is one example
        gathered_logits = gathered_logits.view(1, -1)
        logits = logits.view(1, -1)
 
        # numerator = log(sum(exp(gathered logits)))
        log_score = torch.logsumexp(gathered_logits, dim=dim, keepdim=False)
 
        log_norm = torch.logsumexp(logits, dim=dim, keepdim=False)
        
        # compute the loss
        loss = -(log_score - log_norm) 
        
        # some of the examples might have a loss of `inf` when `target` is all `ignore_index`: when computing start_loss and end_loss for question with the gold answer of yes/no 
        # when `target` is all `ignore_index`, loss is 0 
        loss = loss[~torch.isinf(loss)].sum()
#         loss = torch.tanh(loss)
#         print("final loss: " + str(loss)) 
        return loss 

In [18]:
# debug
# input_ids = torch.tensor([[-1, 5, -1, 2]])
# input_ids.size(0)
# token_indices =  torch.nonzero(input_ids == torch.tensor(-1))[:,1]
# # token_indices
# # token_indices.item()
# # indices =  torch.LongTensor([[2],[0,2]])

# # torch.gather(input_ids, 1, token_indices.unsqueeze(0))
# # p_index = token_indices.view(input_ids.size(0), -1)[:,1::2]   
# # attention_mask = torch.ones(input_ids.shape, dtype=torch.long) 
# # attention_mask[:,token_indices] = 2
# # attention_mask
# p_index = torch.tensor([1, 3, 4])
# s_index = torch.tensor([1,3,6])
# torch.sort(torch.cat((s_index, p_index)))[0]
# attention_mask.view(-1)[ p_index.view(-1), :].view(attention_mask.size(0), -1)
# # for pi in p_index[0]:
# #     attention_mask[:, pi] = 2
# # attention_mask
# # s_index = torch.tensor([[1,3]])
# # torch.sort(torch.cat((p_index, s_index), -1), -1)

# sequence_output  = torch.tensor([[[-1, 5, -1, 2],
#                                  [-2, 27, 2, 9],
#                                  [3, 6, 1, 65],
#                                  [52, 36, 13, 2],
#                                  [73, 26, 1, 7]
#                                 ]])

# sp_para_output_t   = torch.tensor([[[-1],
#                                  [-2 ],
#                                  [3],
#                                  [52],
#                                  [73]
#                                 ]])
# torch.zeros(sp_para_output_t.shape, dtype=torch.float) 

# print("size of sequence_output: " + str(sequence_output.size()))
# # print("size of p_index.unsqueeze(0).unsqueeze(-1): " + str(p_index.unsqueeze(0).size()))
# sequence_output[:,p_index,:]
# b = torch.tensor([0, 1, 2, 3])
# p_index.unsqueeze(-1) * b

# input_ids = torch.tensor([[0.2, 0.0, 0.6, 0.6], [0.2, 0.6, 0.0, 0.0]]) 
# # input_ids.tolist()
# p_index =  torch.nonzero(input_ids == torch.tensor(0.2))
# print(p_index)
# s_index =  torch.nonzero(input_ids == torch.tensor(0.6))
# print(s_index)

# sp_sent = torch.tensor([[0, 1, 1, 0]])
# torch.nonzero(sp_sent, as_tuple=True)[1]
# cat_index = torch.tensor([])
# cat_index = torch.cat((cat_index, ids[0][1]))
# print(ids)
# print(cat_index)
# p_index[p_index[:,0] == 0]

# cat_index[cat_index[:,0].argsort()]

# sorted(torch.cat((p_index, s_index)), key = lambda x: x[0])
# torch.sort(torch.cat((p_index, s_index)), 0)[0]
# for cor in token_indices:
#     attention_mask[cor[0].item()][cor[1].item()] = 2
# attention_mask 
# input_ids = torch.tensor([[-1, 5, -6, 2]])
# print(input_ids.size())
# input_ids.topk(k=2, dim=-1).indices

# predict_type = torch.tensor([[-0.0925, -0.0999, -0.1671]])
# p_type = torch.argmax(predict_type, dim=1).item()
# p_type_score = torch.max(predict_type, dim=1)[0].item()
# print("predict_type: ", predict_type)
# print("p_type: ", p_type)
# print("p_type_score: ", p_type_score)
    
# a = torch.tensor([[0.9213,  1.0887, -0.8858, -1.7683]])
# a.view(-1).size() 
# print(torch.sigmoid(a))
# a = torch.tensor([ 9.213,  1.0887, -0.8858, 7683])
# print(torch.sigmoid(a))

# a = torch.tensor([[[1],[2],[4],[-1],[-1]]])
# a= a.squeeze(-1)
# a.size() 
# a[:, torch.where(a!=-1)[1]]
# m = torch.nn.Sigmoid()
# print("m: ", m)
# loss = torch.nn.BCELoss()
# # input = torch.randn(3, requires_grad=True)
# # print("input: ", input)
# # target = torch.empty(3).random_(2)
# # print("target: ", target)
# # output = loss(m(input), target)
# # print("output: ", output)

# input = torch.tensor([1.0293, -0.1585,  1.1408], requires_grad=True)
# print("input: ", input)
# print("Sigmoid(input): ", m(input))
# target = torch.tensor([0., 1., 0.])
# print("target: ", target)
# output = loss(m(input), target)
# print("output: ", output)

# input = torch.tensor([[1.0293, -0.1585,  1.1408]], requires_grad=True)
# print("input: ", input)
# target = torch.tensor([[0., 1., 0.]])
# print("target: ", target)
# output = loss(m(input), target)
# print("output: ", output)

# 1.1761 * 3
# soft_input = torch.nn.Softmax(dim=-1)
# log_soft_input = torch.log(soft_input(input))
# loss=torch.nn.NLLLoss() 
# loss(log_soft_input, target)
# input = torch.log(soft_input(input))
# loss=torch.nn.NLLLoss()
# loss(input,target)

# loss =torch.nn.CrossEntropyLoss()
# loss(input,target) 

# sp_sent_logits =torch.tensor([[[0.0988],
#          [0.0319],
#          [0.0314]]])
# sp_sent_logits.squeeze()

# input_ids = torch.tensor([[0.6, 0.0, 0.6, 0.0]]) 
# token_indices =  torch.nonzero(input_ids == torch.tensor(0.6))
# token_indices[:,1][0].item()

# def or_softmax_cross_entropy_loss_one_doc(logits, target, ignore_index=-1, dim=-1):
#     """loss function suggested in section 2.2 here https://arxiv.org/pdf/1710.10723.pdf"""
#     assert logits.ndim == 2
#     assert target.ndim == 2
#     assert logits.size(0) == target.size(0) 

#     # with regular CrossEntropyLoss, the numerator is only one of the logits specified by the target, considing only one correct target 
#     # here, the numerator is the sum of a few potential targets, where some of them is the correct answer, considing more correct targets

#     # target are indexes of tokens, padded with ignore_index=-1
#     # logits are scores (one for each label) for each token
# #         print("or_softmax_cross_entropy_loss_one_doc" ) 
# #         print("size of logits: " + str(logits.size()))                    # torch.Size([1, 746]), 746 is number of all tokens 
# #         print("size of target: " + str(target.size()))                    # torch.Size([1, 64]),  -1 padded
#     print("target: " + str(target)) 

#     # compute a target mask
#     target_mask = target == ignore_index
#     # replaces ignore_index with 0, so `gather` will select logit at index 0 for the masked targets
#     masked_target = target * (1 - target_mask.long())                 # replace all -1 in target with 0， tensor([[447,   0,   0,   0, ...]])
#     print("masked_target: " + str(masked_target))     
#     # gather logits
#     gathered_logits = logits.gather(dim=dim, index=masked_target)     # tensor([[0.4382, 0.2340, 0.2340, 0.2340 ... ]]), padding logits are all replaced by logits[0] 
# #         print("size of gathered_logits: " + str(gathered_logits.size()))  # torch.Size([1, 64])
#     print("gathered_logits: " + str(gathered_logits)) 
#     # Apply the mask to gathered_logits. Use a mask of -inf because exp(-inf) = 0
#     gathered_logits[target_mask] = float('-inf')                      # padding logits are all replaced by -inf
#     print("gathered_logits after -inf: " + str(gathered_logits))      # tensor([[0.4382,   -inf,   -inf,   -inf,   -inf,...]])

#     # each batch is one example
#     gathered_logits = gathered_logits.view(1, -1)
#     logits = logits.view(1, -1)
# #         print("size of gathered_logits after view: " + str(gathered_logits.size()))  # torch.Size([1, 64])
# #         print("size of logits after view: " + str(logits.size()))                    # torch.Size([1, 746])　　

#     # numerator = log(sum(exp(gathered logits)))
#     log_score = torch.logsumexp(gathered_logits, dim=dim, keepdim=False)
#     print("log_score: " + str(log_score)) 
#     # denominator = log(sum(exp(logits)))
#     log_norm = torch.logsumexp(logits, dim=dim, keepdim=False)
#     print("log_norm: " + str(log_norm)) 

#     # compute the loss
#     loss = -(log_score - log_norm)
#     print("loss: " + str(loss))


#     # some of the examples might have a loss of `inf` when `target` is all `ignore_index`: when computing start_loss and end_loss for question with the gold answer of yes/no 
#     # replace -inf with 0
#     loss = loss[~torch.isinf(loss)].sum()
#     print("final loss: " + str(loss)) 
#     return loss 

# # input = torch.tensor([[ 0,  0.0780],
# #         [0, 0.9253 ],
# #         [0, 0.0987]])
# # target = torch.tensor([0,1,0])
# # target.size(0) < 1
# # input = torch.tensor([[ 1.1879,  1.0780,  0.5312],
# #         [-0.3499, -1.9253, -1.5725],
# #         [-0.6578, -0.0987,  1.1570]])
# # target=torch.tensor([0,1,2])
# # predict_support_para.view(-1, 2), sp_para.view(-1)
# # input = torch.tensor([[ 1.1879,  1.0780,  0.5312]])
# # target=torch.tensor([0])
# # or_softmax_cross_entropy_loss_one_doc(input, target.unsqueeze(-1))
# # target=torch.tensor([1])
# # or_softmax_cross_entropy_loss_one_doc(input, target.unsqueeze(-1))
# # target=torch.tensor([2])
# # or_softmax_cross_entropy_loss_one_doc(input, target.unsqueeze(-1))
# # target=torch.tensor([-1])
# # or_softmax_cross_entropy_loss_one_doc(input, target.unsqueeze(-1))
# a = torch.tensor([6.4062])    
# b = torch.tensor([2.23])
# torch.cat((a,b))
 
# for a in list_tensor
# from functools import reduce
# reduce(lambda x,y: torch.cat((x,y)), list_tensor[:-1])

# torch.tanh(a)
# # if(torch.isinf(a)):
# #     print("is inf")
# 5 * 1e-2


# import torch
# special_tokens = [1,2]
# input_ids = torch.tensor([[ 1, 0, 2, 1, 0, 2]])

# mask = input_ids != input_ids # initilaize 
# for special_token in special_tokens:
#     mask = torch.logical_or(mask, input_ids.eq(special_token)) 
#     print("mask: ", mask)
# torch.nonzero(mask)    
 

In [19]:
# # debug: check loaded dataset by DataLoader
# tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
# num_new_tokens = tokenizer.add_special_tokens({"additional_special_tokens": ["<p>", "<q>", "</q>"]})
# # # # print(tokenizer.all_special_tokens)    
# # # # print(tokenizer.all_special_ids)     
# # # # tokenizer.convert_tokens_to_ids("<s>")
# # # # tokenizer.sep_token
# print(tokenizer.tokenize("yes"))
# print(tokenizer.tokenize("no"))
# print(tokenizer.tokenize("null"))
# # # all_doc_tokens = []
# # # orig_to_tok_index = []
# # # tok_to_orig_index = []
# # # for (i, token) in enumerate(["<s>", "da", "tell", "<p>", "say"]):
# # #     orig_to_tok_index.append(len(all_doc_tokens))
# # #     sub_tokens = tokenizer.tokenize(f'. {token}')[1:] if i > 0 else tokenizer.tokenize(token)
# # #     for sub_token in sub_tokens:
# # #         tok_to_orig_index.append(i)
# # #         all_doc_tokens.append(sub_token)
# # # all_doc_tokens


 

# dataset = hotpotqaDataset(file_path= args.train_dataset, tokenizer=tokenizer,
#                           max_seq_len= args.max_seq_len, max_doc_len= args.max_doc_len,
#                           doc_stride= args.doc_stride,
#                           max_num_answers= args.max_num_answers,
#                           max_question_len= args.max_question_len,
#                           ignore_seq_with_no_answers= args.ignore_seq_with_no_answers)
# print(len(dataset))

# # # dl = DataLoader(dataset, batch_size=1, shuffle=None,
# # #                     num_workers=args.num_workers, sampler=None,
# # #                     collate_fn=hotpotqaDataset.collate_one_doc_and_lists)

# example = dataset[3]  
# [input_ids, input_mask, segment_ids, subword_starts, subword_ends, q_type, sp_sent, sp_para, qids] = example
 

# print(input_ids[0][:20].tolist())
# print(input_mask) 
# print(segment_ids) 
# print(subword_starts) 
# print(subword_ends)
# print(q_type)
# print(sp_sent) 
# print(sp_para) 
# print(qids)
# print(tokenizer.convert_ids_to_tokens(input_ids[0][667:669+1].tolist()))
# 0.0033 * 90447 
# 28*4
# torch.tensor(0.0)

##### configure_ddp

In [20]:
 %%add_to hotpotqa
 # A hook to overwrite to define your own DDP(DistributedDataParallel) implementation init. 
 # The only requirement is that: 
 # 1. On a validation batch the call goes to model.validation_step.
 # 2. On a training batch the call goes to model.training_step.
 # 3. On a testing batch, the call goes to model.test_step
 def configure_ddp(self, model, device_ids):
    model = LightningDistributedDataParallel(
        model,
        device_ids=device_ids,
        find_unused_parameters=True
    )
    return model

##### **configure_optimizers**

In [21]:
%%add_to hotpotqa
def configure_optimizers(self):
    # Set up optimizers and (optionally) learning rate schedulers
    def lr_lambda(current_step):
        if current_step < self.args.warmup:
            return float(current_step) / float(max(1, self.args.warmup))
        return max(0.0, float(self.args.steps - current_step) / float(max(1, self.args.steps - self.args.warmup)))

    optimizer = torch.optim.Adam(self.parameters(), lr=self.args.lr)

    self.scheduler = LambdaLR(optimizer, lr_lambda, last_epoch=-1)  # scheduler is not saved in the checkpoint, but global_step is, which is enough to restart
    self.scheduler.step(self.global_step)
    print("global step: ", self.global_step)
    return optimizer

##### optimizer_step

In [22]:
%%add_to hotpotqa
# A hook to do a lot of non-standard training tricks such as learning-rate warm-up
def optimizer_step(self, current_epoch, batch_nb, optimizer, optimizer_i, second_order_closure=None,using_native_amp=None):
    optimizer.step()
    optimizer.zero_grad()
    self.scheduler.step(self.global_step)

##### **training_step**

In [23]:
%%add_to hotpotqa
def training_step(self, batch, batch_nb):
    # do the forward pass and calculate the loss for a batch 
    input_ids, input_mask, subword_starts, subword_ends, q_type, qid, answer = batch 
#     print("input_ids: " + str(input_ids)) 
    # print("size of input_ids: " + str(input_ids.size())) 
    output = self.forward(input_ids, subword_starts, subword_ends, q_type)
    answer_loss, type_loss = output[:2]
#     print(self.explainer(input_ids))
#     answer_loss, type_loss, sp_para_loss, sp_sent_loss  = output[:4]
    # print("answer_loss: ", answer_loss)
    # print("type_loss: ", type_loss)
    # print("sp_para_loss: ", sp_para_loss)
    # print("sp_sent_loss: ", sp_sent_loss)

    loss = answer_loss + 5 * type_loss #+ sp_para_loss + sp_sent_loss
#     print("weighted loss: ", loss)
#     print("self.trainer.optimizers[0].param_groups[0]['lr']: ", self.trainer.optimizers[0].param_groups[0]['lr'])
    lr = loss.new_zeros(1) + self.trainer.optimizers[0].param_groups[0]['lr']  # loss.new_zeros(1) is tensor([0.]), converting 'lr' to tensor' by adding it.  

    tensorboard_logs = {'loss': loss, 'train_answer_loss': answer_loss, 'train_type_loss': type_loss,
                        'lr': lr,
                        'mem': torch.tensor(torch.cuda.memory_allocated(input_ids.device) / 1024 ** 3).type_as(loss) }
    return tensorboard_logs


##### training_end

In [24]:
# %%add_to hotpotqa
    # # the function is called for each batch after every epoch is completed
    # def training_end(self, output): 
    #     # print("training_end at epoch: ", self.current_epoch)
    # #     print("len(outputs): ",len(outputs))
    # #     print("output: ",output)
    
    #     # one batch only has one example
    #     avg_loss = output['loss']    
    #     avg_answer_loss = output['train_answer_loss']  
    #     avg_type_loss = output['train_type_loss']    
    #     avg_sp_para_loss = output['train_sp_para_loss']   
    #     avg_sp_sent_loss = output['train_sp_sent_loss'] 
    #     avg_lr = output['lr']      
         
     
    #     if self.trainer.use_ddp:
    #         torch.distributed.all_reduce(avg_loss, op=torch.distributed.ReduceOp.SUM)
    #         avg_loss /= self.trainer.world_size 
    #         torch.distributed.all_reduce(avg_answer_loss, op=torch.distributed.ReduceOp.SUM)
    #         avg_answer_loss /= self.trainer.world_size 
    #         torch.distributed.all_reduce(avg_type_loss, op=torch.distributed.ReduceOp.SUM)
    #         avg_type_loss /= self.trainer.world_size 
    #         torch.distributed.all_reduce(avg_sp_para_loss, op=torch.distributed.ReduceOp.SUM)
    #         avg_sp_para_loss /= self.trainer.world_size 
    #         torch.distributed.all_reduce(avg_sp_sent_loss, op=torch.distributed.ReduceOp.SUM)
    #         avg_sp_sent_loss /= self.trainer.world_size 
    #         torch.distributed.all_reduce(avg_lr, op=torch.distributed.ReduceOp.SUM)
    #         avg_lr /= self.trainer.world_size 
            
     
    #     tensorboard_logs = { #'avg_train_loss': avg_loss, 
    #             'avg_train_answer_loss': avg_answer_loss, 'avg_train_type_loss': avg_type_loss, 'avg_train_sp_para_loss': avg_sp_para_loss, 'avg_train_sp_sent_loss': avg_sp_sent_loss, 'lr': avg_lr
    #           }
    
    #     return {'loss': avg_loss, 'log': tensorboard_logs}


##### validation_step

In [25]:
%%add_to hotpotqa
# When the validation_step is called, the model has been put in eval mode and PyTorch gradients have been disabled. At the end of validation, model goes back to training mode and gradients are enabled.
def validation_step(self, batch, batch_nb):
    input_ids, input_mask, subword_starts, subword_ends, q_type, qid, answer = batch 
    output = self.forward(input_ids, subword_starts, subword_ends, q_type)
    answer_loss, type_loss, start_logits, end_logits, type_logits = output
#     print("start_logits: ", start_logits)
#     print("end_logits: ", end_logits)
    answers_pred = self.decode(input_ids, start_logits, end_logits, type_logits)
    loss = answer_loss + 5*type_loss 

    if(len(answers_pred) != 1):
        print("len(answers_pred) != 1")
        assert(len(answers_pred) == 1)

    pre_answer_score = answers_pred[0]['score']  # (start_logit + end_logit + p_type_score) / 3
    pre_answer = _normalize_text(answers_pred[0]['text'])
    print("pred answer_score: " + str(pre_answer_score))
    print("pred answer_text: " + str(pre_answer)) 

    gold_answer = _normalize_text(answer)
    f1, prec, recall = self.f1_score(pre_answer, gold_answer)
    em = self.exact_match_score(pre_answer, gold_answer) 
    f1 = torch.tensor(f1).type_as(loss)
    prec = torch.tensor(prec).type_as(loss)
    recall = torch.tensor(recall).type_as(loss)
    em = torch.tensor(em).type_as(loss)
#         print("f1: " + str(f1))
#         print("prec: " + str(prec))
#         print("recall: " + str(recall))
#         print("em: " + str(em))  

#     if(len(sp_sent_pred) > 0):
#         sp_sent_em, sp_sent_precision, sp_sent_recall, sp_sent_f1 = self.sp_metrics(sp_sent_pred, torch.where(sp_sent.squeeze())[0].tolist())
#         sp_sent_em = torch.tensor(sp_sent_em).type_as(loss)
#         sp_sent_precision = torch.tensor(sp_sent_precision).type_as(loss)
#         sp_sent_recall = torch.tensor(sp_sent_recall).type_as(loss)
#         sp_sent_f1 = torch.tensor(sp_sent_f1).type_as(loss)

# #         print("sp_sent_em: " + str(sp_sent_em))
# #         print("sp_sent_precision: " + str(sp_sent_precision))
# #         print("sp_sent_recall: " + str(sp_sent_recall))    
# #         print("sp_sent_f1: " + str(sp_sent_f1))    

#         joint_prec = prec * sp_sent_precision
#         joint_recall = recall * sp_sent_recall
#         if joint_prec + joint_recall > 0:
#             joint_f1 = 2 * joint_prec * joint_recall / (joint_prec + joint_recall)
#         else:
#             joint_f1 = torch.tensor(0.0).type_as(loss)
#         joint_em = em * sp_sent_em 

#     else:
#         sp_sent_em, sp_sent_precision, sp_sent_recall, sp_sent_f1 = torch.tensor(0.0).type_as(loss), torch.tensor(0.0).type_as(loss), torch.tensor(0.0).type_as(loss), torch.tensor(0.0).type_as(loss)
#         joint_em, joint_f1, joint_prec, joint_recall =  torch.tensor(0.0).type_as(loss), torch.tensor(0.0).type_as(loss), torch.tensor(0.0).type_as(loss), torch.tensor(0.0).type_as(loss)


    return { 'vloss': loss, 'answer_loss': answer_loss, 'type_loss': type_loss, # 'sp_para_loss': sp_para_loss, 'sp_sent_loss': sp_sent_loss,
               'answer_score': pre_answer_score, 'f1': f1, 'prec':prec, 'recall':recall, 'em': em #,
            #   'sp_em': sp_sent_em, 'sp_f1': sp_sent_f1, 'sp_prec': sp_sent_precision, 'sp_recall': sp_sent_recall,
           #    'joint_em': joint_em, 'joint_f1': joint_f1, 'joint_prec': joint_prec, 'joint_recall': joint_recall
           }


###### decode

In [26]:
%%add_to hotpotqa
def decode(self, input_ids, start_logits, end_logits, type_logits):
#         print("decode")

    question_end_index = self._get_special_index(input_ids, [self.tokenizer.sep_token])[0]
#     print("question_end_index: ", question_end_index)

    # one example per batch
    start_logits = start_logits.squeeze()
    end_logits = end_logits.squeeze()
#     print("start_logits: ", start_logits)
#     print("end_logits: ", end_logits)
    start_logits_indices = start_logits.topk(k=self.args.n_best_size, dim=-1).indices
#     print("start_logits_indices: ", start_logits_indices)
    end_logits_indices = end_logits.topk(k=self.args.n_best_size, dim=-1).indices 
    if(len(start_logits_indices.size()) > 1):
        print("len(start_logits_indices.size()): ", len(start_logits_indices.size()))
        assert("len(start_logits_indices.size()) > 1")
    p_type = torch.argmax(type_logits, dim=1).item()
    p_type_score = torch.max(type_logits, dim=1)[0] 
#     print("type_logits: ", type_logits)
#         print("p_type: ", p_type)
#         print("p_type_score: ", p_type_score)

    answers = []
    if p_type == 0:
        potential_answers = []
        for start_logit_index in start_logits_indices: 
            for end_logit_index in end_logits_indices: 
                if start_logit_index <= question_end_index.item():
                    continue
                if end_logit_index <= question_end_index.item():
                    continue
                if start_logit_index > end_logit_index:
                    continue
                answer_len = end_logit_index - start_logit_index + 1
                if answer_len > self.args.max_answer_length:
                    continue
                potential_answers.append({'start': start_logit_index, 'end': end_logit_index,
                                          'start_logit': start_logits[start_logit_index],  # single logit score for start position at start_logit_index
                                          'end_logit': end_logits[end_logit_index]})    
        sorted_answers = sorted(potential_answers, key=lambda x: (x['start_logit'] + x['end_logit']), reverse=True) 
#         print("sorted_answers: " + str(sorted_answers))

        if len(sorted_answers) == 0:
            answers.append({'text': 'null', 'score': 1})
        else:
            answer = sorted_answers[0]
            answer_token_ids = input_ids[0, answer['start']: answer['end'] + 1]

            answer_tokens = self.tokenizer.convert_ids_to_tokens(answer_token_ids.tolist())

            # remove </s>
            special_token = self.tokenizer.sep_token 
            try:
                if(answer_tokens[0] == special_token):
                    answer['start_logit'] = -2000000
                elif(answer_tokens[-1] == special_token):
                    answer['end_logit'] = -2000000
                answer_tokens.remove(special_token)
            except:
                pass

            text = self.tokenizer.convert_tokens_to_string(answer_tokens)
#                 score = (answer['start_logit'] + answer['end_logit'] + p_type_score) / 3
            score = (torch.sigmoid(answer['start_logit']) + torch.sigmoid(answer['end_logit']) + torch.sigmoid(p_type_score)) / 3
            answers.append({'text': text, 'score': score})
            print("pred answers: " + str(answers))
    elif p_type == 1: 
        answers.append({'text': 'yes', 'score': p_type_score})
    elif p_type == 2:
        answers.append({'text': 'no', 'score': p_type_score})
    elif p_type == 3:
        answers.append({'text': 'null', 'score': p_type_score})
    else:
        assert False 


#     sent_indexes = self._get_special_index(input_ids, [SENT_MARKER_END])
#     para_indexes = self._get_special_index(input_ids, [TITLE_START])

#     s_to_p_map = []
#     for s in sent_indexes:
#         s_to_p = torch.where(torch.le(para_indexes, s))[0][-1]     # last para_index smaller or equal to s
#         s_to_p_map.append(s_to_p.item()) 
# #         print("s_to_p_map: " + str(s_to_p_map))

# #         print("sp_para_logits", sp_para_logits)
# #         print("sp_sent_logits", sp_sent_logits)

#     sp_para_top2 = sp_para_logits.squeeze().topk(k=2).indices
#     if(sp_sent_logits.squeeze().size(0) > 12):
#         sp_sent_top12 = sp_sent_logits.squeeze().topk(k=12).indices
#     else:
#         sp_sent_top12 = sp_sent_logits.squeeze().topk(k=sp_sent_logits.squeeze().size(0)).indices
# #         print("sp_para_top2", sp_para_top2)
# #         print("sp_sent_top12", sp_sent_top12)

#     sp_sent_pred = set()
#     sp_para_pred = set(sp_para_top2.tolist())
#     for sp_sent in sp_sent_top12:
#         sp_sent_to_para = s_to_p_map[sp_sent.item()]
#         if sp_sent_to_para in sp_para_top2:
#             sp_sent_pred.add(sp_sent.item())
# #             sp_para_pred.add(sp_sent_to_para) 
# #         print("sp_sent_pred: " + str(sp_sent_pred))
# #         print("sp_para_pred: " + str(sp_para_pred))
#     return (answers, sp_sent_pred, sp_para_pred)
    return answers

###### metrics

In [27]:
%%add_to hotpotqa
# def _normalize_text(self, s):

#     def remove_articles(text):
#         return re.sub(r'\b(a|an|the)\b', ' ', text)

#     def white_space_fix(text):
#         return ' '.join(text.split())

#     def remove_punc(text):
#         exclude = set(string.punctuation)
#         return ''.join(ch for ch in text if ch not in exclude)

#     def lower(text):
#         return text.lower()

#     return white_space_fix(remove_articles(remove_punc(lower(s))))


def f1_score(self, prediction, ground_truth):
    normalized_prediction = _normalize_text(prediction)
    normalized_ground_truth = _normalize_text(ground_truth)
    ZERO_METRIC = (0, 0, 0)

    if normalized_prediction in ['yes', 'no', 'noanswer'] and normalized_prediction != normalized_ground_truth:
        return ZERO_METRIC
    if normalized_ground_truth in ['yes', 'no', 'noanswer'] and normalized_prediction != normalized_ground_truth:
        return ZERO_METRIC

    prediction_tokens = normalized_prediction.split()
    ground_truth_tokens = normalized_ground_truth.split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return ZERO_METRIC
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1, precision, recall


def exact_match_score(self, prediction, ground_truth):
    return int(_normalize_text(prediction) == _normalize_text(ground_truth))


def sp_metrics(self, prediction, gold): 
    tp, fp, fn = 0, 0, 0
    for e in prediction:
        if e in gold:
            tp += 1
        else:
            fp += 1 
    for e in gold:
        if e not in prediction:
            fn += 1 
    prec = 1.0 * tp / (tp + fp) if tp + fp > 0 else 0.0
    recall = 1.0 * tp / (tp + fn) if tp + fn > 0 else 0.0
    f1 = 2 * prec * recall / (prec + recall) if prec + recall > 0 else 0.0
    em = 1.0 if fp + fn == 0 else 0.0 
    return em, prec, recall, f1 



##### validation_end

In [28]:
%%add_to hotpotqa
# If a validation_step is not defined, this won't be called. Called at the end of the validation loop with the outputs of validation_step.
def validation_end(self, outputs):
    print("validation_end")
    avg_loss = torch.stack([x['vloss'] for x in outputs]).mean()  
    avg_answer_loss = torch.stack([x['answer_loss'] for x in outputs]).mean()  
    avg_type_loss = torch.stack([x['type_loss'] for x in outputs]).mean()  
#     avg_sp_para_loss = torch.stack([x['sp_para_loss'] for x in outputs]).mean()  
#     avg_sp_sent_loss = torch.stack([x['sp_sent_loss'] for x in outputs]).mean()  


    answer_scores = [x['answer_score'] for x in outputs] 
    f1_scores = [x['f1'] for x in outputs]  
    em_scores = [x['em'] for x in outputs]  
    prec_scores =  [x['prec'] for x in outputs] 
    recall_scores = [x['recall'] for x in outputs]  
#     sp_sent_f1_scores = [x['sp_f1'] for x in outputs]   
#     sp_sent_em_scores = [x['sp_em'] for x in outputs]   
#     sp_sent_prec_scores = [x['sp_prec'] for x in outputs]   
#     sp_sent_recall_scores = [x['sp_recall'] for x in outputs]   
#     joint_f1_scores = [x['joint_f1'] for x in outputs]  
#     joint_em_scores = [x['joint_em'] for x in outputs]  
#     joint_prec_scores = [x['joint_prec'] for x in outputs]  
#     joint_recall_scores = [x['joint_recall'] for x in outputs]



    print(f'before sync --> sizes:  {len(answer_scores)}, {len(f1_scores)}, {len(em_scores)}')
    if self.trainer.use_ddp:
        torch.distributed.all_reduce(avg_loss, op=torch.distributed.ReduceOp.SUM)
        avg_loss /= self.trainer.world_size 
        torch.distributed.all_reduce(avg_answer_loss, op=torch.distributed.ReduceOp.SUM)
        avg_answer_loss /= self.trainer.world_size 
        torch.distributed.all_reduce(avg_type_loss, op=torch.distributed.ReduceOp.SUM)
        avg_type_loss /= self.trainer.world_size 
#         torch.distributed.all_reduce(avg_sp_para_loss, op=torch.distributed.ReduceOp.SUM)
#         avg_sp_para_loss /= self.trainer.world_size 
#         torch.distributed.all_reduce(avg_sp_sent_loss, op=torch.distributed.ReduceOp.SUM)
#         avg_sp_sent_loss /= self.trainer.world_size 

        answer_scores = self.sync_list_across_gpus(answer_scores, avg_loss.device, torch.float)
        f1_scores = self.sync_list_across_gpus(f1_scores, avg_loss.device, torch.float)
        em_scores = self.sync_list_across_gpus(em_scores, avg_loss.device, torch.float)
        prec_scores = self.sync_list_across_gpus(prec_scores, avg_loss.device, torch.float)
        recall_scores = self.sync_list_across_gpus(recall_scores, avg_loss.device, torch.float)

#         sp_sent_f1_scores = self.sync_list_across_gpus(sp_sent_f1_scores, avg_loss.device, torch.float)
#         sp_sent_em_scores = self.sync_list_across_gpus(sp_sent_em_scores, avg_loss.device, torch.float)
#         sp_sent_prec_scores = self.sync_list_across_gpus(sp_sent_prec_scores, avg_loss.device, torch.float)
#         sp_sent_recall_scores = self.sync_list_across_gpus(sp_sent_recall_scores, avg_loss.device, torch.float)

#         joint_f1_scores = self.sync_list_across_gpus(joint_f1_scores, avg_loss.device, torch.float)
#         joint_em_scores = self.sync_list_across_gpus(joint_em_scores, avg_loss.device, torch.float)
#         joint_prec_scores = self.sync_list_across_gpus(joint_prec_scores, avg_loss.device, torch.float)
#         joint_recall_scores = self.sync_list_across_gpus(joint_recall_scores, avg_loss.device, torch.float)


    print(f'after sync --> sizes: {len(answer_scores)}, {len(f1_scores)}, {len(em_scores)}')

    avg_val_f1 = sum(f1_scores) / len(f1_scores)    
    avg_val_em = sum(em_scores) / len(em_scores)    
    avg_val_prec = sum(prec_scores) / len(prec_scores)  
    avg_val_recall = sum(recall_scores) / len(recall_scores)    
#     avg_val_sp_sent_f1 = sum(sp_sent_f1_scores) / len(sp_sent_f1_scores)    
#     avg_val_sp_sent_em = sum(sp_sent_em_scores) / len(sp_sent_em_scores)    
#     avg_val_sp_sent_prec = sum(sp_sent_prec_scores) / len(sp_sent_prec_scores)  
#     avg_val_sp_sent_recall = sum(sp_sent_recall_scores) / len(sp_sent_recall_scores)    
#     avg_val_joint_f1 = sum(joint_f1_scores) / len(joint_f1_scores)  
#     avg_val_joint_em = sum(joint_em_scores) / len(joint_em_scores)  
#     avg_val_joint_prec = sum(joint_prec_scores) / len(joint_prec_scores)    
#     avg_val_joint_recall = sum(joint_recall_scores) / len(joint_recall_scores)  

    print("avg_loss: ", avg_loss, end = '\t')   
    print("avg_answer_loss: ", avg_answer_loss, end = '\t') 
    print("avg_type_loss: ", avg_type_loss, end = '\t') 
#     print("avg_sp_para_loss: ", avg_sp_para_loss, end = '\t')   
#     print("avg_sp_sent_loss: ", avg_sp_sent_loss)   
    print("avg_val_f1: ", avg_val_f1, end = '\t')   
    print("avg_val_em: ", avg_val_em, end = '\t')   
    print("avg_val_prec: ", avg_val_prec, end = '\t')   
    print("avg_val_recall: ", avg_val_recall)   
#     print("avg_val_sp_sent_f1: ", avg_val_sp_sent_f1, end = '\t')   
#     print("avg_val_sp_sent_em: " , avg_val_sp_sent_em, end = '\t')  
#     print("avg_val_sp_sent_prec: ", avg_val_sp_sent_prec, end = '\t')   
#     print("avg_val_sp_sent_recall: ", avg_val_sp_sent_recall)   
#     print("avg_val_joint_f1: " , avg_val_joint_f1, end = '\t')  
#     print("avg_val_joint_em: ", avg_val_joint_em, end = '\t')   
#     print("avg_val_joint_prec: ", avg_val_joint_prec, end = '\t')   
#     print("avg_val_joint_recall: ", avg_val_joint_recall)   


    logs = {'avg_val_loss': avg_loss, 'avg_val_answer_loss': avg_answer_loss, 'avg_val_type_loss': avg_type_loss, 
            #'avg_val_sp_para_loss': avg_sp_para_loss, 'avg_val_sp_sent_loss': avg_sp_sent_loss,   
    'avg_val_f1': avg_val_f1, 'avg_val_em': avg_val_em,  'avg_val_prec': avg_val_prec, 'avg_val_recall': avg_val_recall#,    
    #'avg_val_sp_sent_f1': avg_val_sp_sent_f1, 'avg_val_sp_sent_em': avg_val_sp_sent_em,  'avg_val_sp_sent_prec': avg_val_sp_sent_prec, 'avg_val_sp_sent_recall': avg_val_sp_sent_recall,    
   # 'avg_val_joint_f1': avg_val_joint_f1, 'avg_val_joint_em': avg_val_joint_em,  'avg_val_joint_prec': avg_val_joint_prec, 'avg_val_joint_recall': avg_val_joint_recall 
    }   

    return {'avg_val_loss': avg_loss, 'log': logs}

def sync_list_across_gpus(self, l, device, dtype):
    l_tensor = torch.tensor(l, device=device, dtype=dtype)
    gather_l_tensor = [torch.ones_like(l_tensor) for _ in range(self.trainer.world_size)]
    torch.distributed.all_gather(gather_l_tensor, l_tensor)
    return torch.cat(gather_l_tensor).tolist()

##### test_step

In [29]:
# %%add_to hotpotqa
# def test_step(self, batch, batch_nb):
#     input_ids, input_mask, segment_ids, subword_starts, subword_ends, q_type, sp_sent, sp_para, qid, answer = batch

#     print("test_step of qid: ", qid, end="\t") 
#     output = self.forward(input_ids, input_mask, segment_ids, subword_starts, subword_ends, q_type, sp_sent, sp_para)
#     answer_loss, type_loss, sp_para_loss, sp_sent_loss, start_logits, end_logits, type_logits, sp_para_output, sp_sent_output = output 
#     loss = answer_loss + 5*type_loss + 10*sp_para_loss + 10*sp_sent_loss

#     answers_pred, sp_sent_pred, sp_para_pred = self.decode(input_ids, start_logits, end_logits, type_logits, sp_para_output, sp_sent_output)

#     if(len(answers_pred) != 1):
#         print("len(answers_pred) != 1")
#         assert(len(answers_pred) == 1)

#     pre_answer_score = answers_pred[0]['score']  # (start_logit + end_logit + p_type_score) / 3
#     pre_answer = _normalize_text(answers_pred[0]['text'])
#     # print("pred answer_score: " + str(pre_answer_score))
#     # print("pred answer_text: " + str(pre_answer)) 

#     gold_answer = _normalize_text(answer)

#     print("pre_answer:\t", pre_answer, "\tgold_answer:\t", gold_answer)

#     return { 'vloss': loss, 'answer_loss': answer_loss, 'type_loss': type_loss, 'sp_para_loss': sp_para_loss, 'sp_sent_loss': sp_sent_loss, 'answer_score': pre_answer_score}


##### test_end

In [30]:
# %%add_to hotpotqa
# def test_end(self, outputs):
#     print("test_end")
#     avg_loss = torch.stack([x['vloss'] for x in outputs]).mean()  
#     avg_answer_loss = torch.stack([x['answer_loss'] for x in outputs]).mean()  
#     avg_type_loss = torch.stack([x['type_loss'] for x in outputs]).mean()  
#     avg_sp_para_loss = torch.stack([x['sp_para_loss'] for x in outputs]).mean()  
#     avg_sp_sent_loss = torch.stack([x['sp_sent_loss'] for x in outputs]).mean()  

#     answer_scores = [x['answer_score'] for x in outputs]  # [item for sublist in outputs for item in sublist['answer_score']] #torch.stack([x['answer_score'] for x in outputs]).mean() # 


#     print(f'before sync --> sizes:  {len(answer_scores)}')
#     if self.trainer.use_ddp:
#         torch.distributed.all_reduce(avg_loss, op=torch.distributed.ReduceOp.SUM)
#         avg_loss /= self.trainer.world_size 
#         torch.distributed.all_reduce(avg_answer_loss, op=torch.distributed.ReduceOp.SUM)
#         avg_answer_loss /= self.trainer.world_size 
#         torch.distributed.all_reduce(avg_type_loss, op=torch.distributed.ReduceOp.SUM)
#         avg_type_loss /= self.trainer.world_size 
#         torch.distributed.all_reduce(avg_sp_para_loss, op=torch.distributed.ReduceOp.SUM)
#         avg_sp_para_loss /= self.trainer.world_size 
#         torch.distributed.all_reduce(avg_sp_sent_loss, op=torch.distributed.ReduceOp.SUM)
#         avg_sp_sent_loss /= self.trainer.world_size 

# #         int_qids = self.sync_list_across_gpus(int_qids, avg_loss.device, torch.int)
#         answer_scores = self.sync_list_across_gpus(answer_scores, avg_loss.device, torch.float)


#     print(f'after sync --> sizes: {len(answer_scores)}')
#     # print("answer_scores: ", answer_scores)

#     # print("avg_loss: ", avg_loss, end = '\t') 
#     # print("avg_answer_loss: ", avg_answer_loss, end = '\t') 
#     # print("avg_type_loss: ", avg_type_loss, end = '\t') 
#     # print("avg_sp_para_loss: ", avg_sp_para_loss, end = '\t') 
#     # print("avg_sp_sent_loss: ", avg_sp_sent_loss, end = '\t')  

#     logs = {'avg_val_loss': avg_loss, 'avg_val_answer_loss': avg_answer_loss, 'avg_val_type_loss': avg_type_loss, 'avg_val_sp_para_loss': avg_sp_para_loss, 'avg_val_sp_sent_loss': avg_sp_sent_loss
#            }

#     return {'avg_val_loss': avg_loss, 'log': logs} 


##### add_model_specific_args

In [31]:
%%add_to hotpotqa
@staticmethod
def add_model_specific_args(parser, root_dir):
    parser.add_argument("--save_dir", type=str, default='jupyter-hotpotqa')
    parser.add_argument("--save_prefix", type=str, required=True)
    parser.add_argument("--train_dataset", type=str, required=False, help="Path to the training squad-format")
    parser.add_argument("--dev_dataset", type=str, required=True, help="Path to the dev squad-format")
    parser.add_argument("--batch_size", type=int, default=2, help="Batch size")
    parser.add_argument("--gpus", type=str, default='0',
                        help="Comma separated list of gpus. Default is gpu 0. To use CPU, use --gpus "" ")
    parser.add_argument("--warmup", type=int, default=1000, help="Number of warmup steps")
    parser.add_argument("--lr", type=float, default=0.00005, help="Maximum learning rate")
    parser.add_argument("--val_every", type=float, default=1.0, help="How often within one training epoch to check the validation set.")
    parser.add_argument("--val_percent_check", default=1.00, type=float, help='Percent of validation data used')
    parser.add_argument("--num_workers", type=int, default=4, help="Number of data loader workers")
    parser.add_argument("--seed", type=int, default=1234, help="Seed")
    parser.add_argument("--epochs", type=int, default=6, help="Number of epochs")
    parser.add_argument("--max_seq_len", type=int, default=4096,
                        help="Maximum length of seq passed to the transformer model")
    parser.add_argument("--max_doc_len", type=int, default=4096,
                        help="Maximum number of wordpieces of the input document")
    parser.add_argument("--max_num_answers", type=int, default=64,
                        help="Maximum number of answer spans per document (64 => 94%)")
    parser.add_argument("--max_question_len", type=int, default=55,
                        help="Maximum length of the question")
    parser.add_argument("--doc_stride", type=int, default=-1,
                        help="Overlap between document chunks. Use -1 to only use the first chunk")
    parser.add_argument("--ignore_seq_with_no_answers", action='store_true',
                        help="each example should have at least one answer. Default is False")
    parser.add_argument("--disable_checkpointing", action='store_true', help="No logging or checkpointing")
    parser.add_argument("--n_best_size", type=int, default=20,
                        help="Number of answer candidates. Used at decoding time")
    parser.add_argument("--max_answer_length", type=int, default=30,
                        help="maximum num of wordpieces/answer. Used at decoding time")
    parser.add_argument("--regular_softmax_loss", action='store_true', help="IF true, use regular softmax. Default is using ORed softmax loss")
    parser.add_argument("--test", action='store_true', help="Test only, no training")
    parser.add_argument("--model_path", type=str,
                        help="Path to the checkpoint directory")
    parser.add_argument("--no_progress_bar", action='store_true', help="no progress bar. Good for printing")
    parser.add_argument("--attention_mode", type=str, choices=['tvm', 'sliding_chunks'],
                        default='sliding_chunks', help='Which implementation of selfattention to use')
    parser.add_argument("--fp32", action='store_true', help="default is fp16. Use --fp32 to switch to fp32")
    parser.add_argument('--train_percent', type=float, default=1.0)
    return parser

##### class info

In [32]:
dir(hotpotqa)

['CHECKPOINT_HYPER_PARAMS_KEY',
 'CHECKPOINT_HYPER_PARAMS_NAME',
 'CHECKPOINT_HYPER_PARAMS_TYPE',
 'T_destination',
 '_LightningModule__get_hparams_assignment_variable',
 '__abstractmethods__',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_cache',
 '_abc_negative_cache',
 '_abc_negative_cache_version',
 '_abc_registry',
 '_apply',
 '_auto_collect_arguments',
 '_call_impl',
 '_forward_unimplemented',
 '_get_name',
 '_get_special_index',
 '_init_slurm_connection',
 '_load_from_state_dict',
 '_load_model_state',
 '_named_members',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',


In [33]:
from inspect import getmembers, isfunction
getmembers(hotpotqa)

[('CHECKPOINT_HYPER_PARAMS_KEY', 'hyper_parameters'),
 ('CHECKPOINT_HYPER_PARAMS_NAME', 'hparams_name'),
 ('CHECKPOINT_HYPER_PARAMS_TYPE', 'hparams_type'),
 ('T_destination', ~T_destination),
 ('_LightningModule__get_hparams_assignment_variable',
  <function pytorch_lightning.core.lightning.LightningModule.__get_hparams_assignment_variable(self)>),
 ('__abstractmethods__', frozenset()),
 ('__annotations__',
  {'_device': Ellipsis, '_dtype': typing.Union[str, torch.dtype]}),
 ('__call__',
  <function torch.nn.modules.module.Module._call_impl(self, *input, **kwargs)>),
 ('__class__', abc.ABCMeta),
 ('__delattr__',
  <function torch.nn.modules.module.Module.__delattr__(self, name)>),
 ('__dict__',
  mappingproxy({'__module__': '__main__',
                '__init__': <function __main__.hotpotqa.__init__(self, args)>,
                'load_model': <function __main__.hotpotqa.load_model(self)>,
                'train_dataloader': <function pytorch_lightning.core.decorators.data_loader.<local

In [34]:
functions_list = [o for o in getmembers(hotpotqa) if isfunction(o[1])]
functions_list

[('_LightningModule__get_hparams_assignment_variable',
  <function pytorch_lightning.core.lightning.LightningModule.__get_hparams_assignment_variable(self)>),
 ('__call__',
  <function torch.nn.modules.module.Module._call_impl(self, *input, **kwargs)>),
 ('__delattr__',
  <function torch.nn.modules.module.Module.__delattr__(self, name)>),
 ('__dir__', <function torch.nn.modules.module.Module.__dir__(self)>),
 ('__getattr__',
  <function torch.nn.modules.module.Module.__getattr__(self, name:str) -> Union[torch.Tensor, _ForwardRef('Module')]>),
 ('__init__', <function __main__.hotpotqa.__init__(self, args)>),
 ('__repr__', <function torch.nn.modules.module.Module.__repr__(self)>),
 ('__setattr__',
  <function torch.nn.modules.module.Module.__setattr__(self, name:str, value:Union[torch.Tensor, _ForwardRef('Module')]) -> None>),
 ('__setstate__',
  <function torch.nn.modules.module.Module.__setstate__(self, state)>),
 ('_apply', <function torch.nn.modules.module.Module._apply(self, fn)>),


In [35]:
import inspect
inspect.getmro(hotpotqa)  # a hierarchy of classes 

(__main__.hotpotqa,
 pytorch_lightning.core.lightning.LightningModule,
 abc.ABC,
 pytorch_lightning.utilities.device_dtype_mixin.DeviceDtypeModuleMixin,
 pytorch_lightning.core.grads.GradInformation,
 pytorch_lightning.core.saving.ModelIO,
 pytorch_lightning.core.hooks.ModelHooks,
 torch.nn.modules.module.Module,
 object)

In [36]:
help(hotpotqa.configure_optimizers)

Help on function configure_optimizers in module __main__:

configure_optimizers(self)



In [37]:
# import inspect
# code, line_no = inspect.getsourcelines(hotpotqa.training_step)
# print(''.join(code))

### main

In [38]:
def main(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(args.seed)

    if not args.test:     # if it needs to train, remove exsiting folder
        import shutil
        save_folder = os.path.join(args.save_dir, args.save_prefix)
        if os.path.exists(save_folder):
            shutil.rmtree(save_folder, ignore_errors=True)  #delete non-empty folder 
        
    import shutil
    save_folder = os.path.join(args.save_dir, args.save_prefix)
    if os.path.exists(save_folder):
        shutil.rmtree(save_folder, ignore_errors=True)  #delete non-empty folder
  
    print("main")

In [39]:
# !git clone https://huggingface.co/lysandre/tiny-longformer-random
# import os 
# os.getcwd()

In [42]:

    hotpotqa.__abstractmethods__=set()   # without this, got an error "Can't instantiate abstract class hotpotqa with abstract methods" if these two abstract methods are not implemented in the same cell where class hotpotqa defined 
    model = hotpotqa(args)
    model.to('cuda')    # this is necessary to use gpu
    

Loaded model with config:
DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "vocab_size": 30522
}



hotpotqa(
  (model): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in_features=

In [43]:
 
    logger = TestTubeLogger( # The TestTubeLogger adds a nicer folder structure to manage experiments and snapshots all hyperparameters you pass to a LightningModule.
        save_dir=args.save_dir,
        name=args.save_prefix,
        version=0  # always use version=0
    )

In [44]:
 
    checkpoint_callback = ModelCheckpoint(
        filepath=os.path.join(args.save_dir, args.save_prefix, "checkpoints"),
        save_top_k=5,
        verbose=True,
        monitor='avg_val_f1',
        mode='max',
        prefix=''
    )

In [45]:
 
    train_set_size = 9 * args.train_percent # 90447 * args.train_percent   # hardcode dataset size. Needed to compute number of steps for the lr scheduler
    print("train_set_size: ", train_set_size) 

    args.gpus = [int(x) for x in args.gpus.split(',')] if args.gpus!='' else None
    num_devices = 1 or len(args.gpus)
    print("num_devices: ", num_devices)

    train_set_size = 90447 * args.train_percent    # hardcode dataset size. Needed to compute number of steps for the lr scheduler
    args.steps = args.epochs * train_set_size / (args.batch_size * num_devices)

    print(f'>>>>>>> #train_set_size: {train_set_size}, #steps: {args.steps},  #warmup steps: {args.warmup}, #epochs: {args.epochs}, batch_size: {args.batch_size * num_devices} <<<<<<<')

train_set_size:  9.0
num_devices:  1
>>>>>>> #train_set_size: 90447.0, #steps: 271341.0,  #warmup steps: 1000, #epochs: 6, batch_size: 2 <<<<<<<


In [46]:
 
    trainer = pl.Trainer(gpus=args.gpus, distributed_backend='ddp', # if args.gpus and (len(args.gpus) > 1) else None,
                             track_grad_norm=-1, max_epochs=args.epochs, early_stop_callback=None,
                             accumulate_grad_batches=args.batch_size,
                             train_percent_check = args.train_percent,
        #                          val_check_interval=args.val_every,
                             val_percent_check=args.val_percent_check,
                             test_percent_check=args.val_percent_check,
                             logger=logger if not args.disable_checkpointing else False,
                             checkpoint_callback=checkpoint_callback if not args.disable_checkpointing else False,
                             show_progress_bar=args.no_progress_bar,
#                              use_amp=True, #not args.fp32, 
                             amp_level='O2',
                             check_val_every_n_epoch=1
                             )


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Multi-processing is handled by Slurm.
CUDA_VISIBLE_DEVICES: [0]


In [47]:
 
    #     if not args.test: 
    trainer.fit(model)


/home/u32/fanluo/.local/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:25: RuntimeWarning: You have defined a `val_dataloader()` and have defined a `validation_step()`, you may also want to define `validation_epoch_end()` for accumulating stats.
  warnings.warn(*args, **kwargs)
initializing ddp: GLOBAL_RANK: 0, MEMBER: 1/1
----------------------------------------------------------------------------------------------------
distributed_backend=ddp
All DDP processes registered. Starting ddp with 1 processes
----------------------------------------------------------------------------------------------------


global step:  0


/home/u32/fanluo/.local/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/home/u32/fanluo/.local/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:143: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate 

reading file: small_3.json


sequence_output  tensor([[[-0.2081,  0.1313,  0.0021,  ..., -0.1503,  0.3243,  0.4670],
         [-0.6907, -0.1512,  0.5700,  ...,  0.0389,  0.3567,  0.3901],
         [-0.4784,  0.0030,  0.7859,  ...,  0.0939,  0.0775,  0.5939],
         ...,
         [ 0.1300,  0.3617,  0.6407,  ..., -0.3245,  0.0267, -0.4984],
         [-0.8131,  0.0707,  0.0075,  ...,  0.0313, -0.1685, -0.4562],
         [ 0.5895,  0.0287,  0.0027,  ...,  0.1481, -0.1669, -0.3316]]],
       device='cuda:0')
pred answers: [{'text': '##ogene boundary k – t boundary diamond lane in united states and canada diamond lane', 'score': tensor([0.6127], device='cuda:0')}]
pred answer_score: tensor([0.6127], device='cuda:0')
pred answer_text: ogene boundary k – t boundary diamond lane in united states and canada diamond lane
sequence_output  tensor([[[-0.3084, -0.0458,  0.0272,  ..., -0.0851,  0.4171,  0.1915],
         [-0.6778, -0.0116, -0.1904,  ...,  0.2765,  0.9211, -0.4125],
         [-0.9655,  0.1330,  0.3725,  ..., -0

...

sequence_output  tensor([[[ 0.1530, -0.6805, -0.6340,  ...,  0.4376,  0.3339, -0.5422],
         [-1.1228, -0.8644,  0.3350,  ...,  0.2478,  1.1911, -0.4329],
         [-0.7444, -0.4888,  0.0098,  ...,  0.6140,  0.3541, -1.9935],
         ...,
         [ 0.5419, -0.5966,  0.6048,  ..., -0.7611,  0.1599, -0.3887],
         [ 0.4550,  0.6304, -0.3518,  ..., -0.1115, -0.5443, -1.1363],
         [ 0.6680, -0.2213, -0.1907,  ...,  0.2883, -0.2629, -0.1465]]],
       device='cuda:0', grad_fn=<NativeLayerNormBackward>)
sequence_output  tensor([[[-0.0247, -0.8225, -0.7019,  ...,  0.3138,  0.4227, -0.5151],
         [-0.9415, -0.4425, -0.0989,  ...,  0.1451,  0.2926, -1.4248],
         [-1.0571, -0.3129,  0.3378,  ...,  0.3150,  0.0848, -0.9195],
         ...,
         [-0.5535, -0.5682,  0.2976,  ...,  0.1229,  0.1467, -0.3482],
         [-0.7822, -0.2969, -0.2737,  ..., -0.2449, -0.1625, -1.7627],
         [ 0.6725, -0.0673,  0.1879,  ...,  0.2266, -0.3249, -0.2137]]],
       device='cuda:0',

sequence_output  tensor([[[ 0.1371, -0.7355, -0.7611,  ...,  0.2172,  0.7229,  0.4144],
         [-1.0908, -0.2585,  0.4821,  ...,  0.0480,  0.4255,  0.5271],
         [-0.6043, -0.1051,  0.9984,  ...,  0.0835,  0.1237,  0.6393],
         ...,
         [-0.5240,  0.0725,  0.2854,  ..., -0.3695, -0.3517, -0.8445],
         [-1.5426, -0.8256, -0.7887,  ...,  0.0276, -0.1862, -0.5750],
         [ 0.7379, -0.4366, -0.0169,  ...,  0.3458, -0.2573, -0.1992]]],
       device='cuda:0')
pred answer_score: tensor([2.3413], device='cuda:0')
pred answer_text: null
sequence_output  tensor([[[-0.0835, -0.7769, -0.8853,  ...,  0.2829,  0.0847, -0.3342],
         [-1.0367, -0.3683, -0.9457,  ...,  0.6232,  1.3266, -0.9772],
         [-1.2948, -0.0451, -0.0429,  ..., -0.0750,  0.5539, -0.0289],
         ...,
         [-0.4958,  0.1678, -0.1211,  ..., -0.0948, -0.3127, -1.4513],
         [-0.3619, -0.5610, -0.2894,  ..., -0.6306, -0.1021, -1.7603],
         [ 0.7948, -0.1350, -0.1848,  ...,  0.2416, -0.


Epoch 00005: avg_val_f1  was not in top 5


validation_end
before sync --> sizes:  10, 10, 10
after sync --> sizes: 10, 10, 10
avg_loss:  tensor(7.5525, device='cuda:0')	avg_answer_loss:  tensor(3.5354, device='cuda:0')	avg_type_loss:  tensor(0.8034, device='cuda:0')	avg_val_f1:  0.0	avg_val_em:  0.0	avg_val_prec:  0.0	avg_val_recall:  0.0



1

In [ ]:
#     trainer.test(model)

In [ ]:
### To install apex ### 
#     !git clone https://github.com/NVIDIA/apex
#     os.chdir("/xdisk/msurdeanu/fanluo/hotpotQA/apex/")
#     !module load cuda101/neuralnet/7/7.6.4  
#     !module load cuda10.1/toolkit/10.1.243 
#     !conda install -c conda-forge cudatoolkit-dev --yes
#     !conda install -c conda-forge/label/cf201901 cudatoolkit-dev --yes
#     !conda install -c conda-forge/label/cf202003 cudatoolkit-dev --yes
#     !which nvcc
#     !python -m pip install -v --no-cache-dir ./
#     os.chdir("/xdisk/msurdeanu/fanluo/hotpotQA/")

In [41]:
# debug: check args
import shlex
argString ='--train_dataset small.json  --dev_dataset small_3.json  \
    --gpus 0 --num_workers 4 \
    --max_seq_len 512 \
    --save_prefix hotpotqa-longformer'
# --train_dataset hotpot_train_v1.1.json  --dev_dataset hotpot_dev_distractor_v1.json  \

import argparse 
# if __name__ == "__main__":
main_arg_parser = argparse.ArgumentParser(description="hotpotqa")
parser = hotpotqa.add_model_specific_args(main_arg_parser, os.getcwd())
args = parser.parse_args(shlex.split(argString)) 
for arg in vars(args):
    print((arg, getattr(args, arg)))
main(args)

('save_dir', 'jupyter-hotpotqa')
('save_prefix', 'hotpotqa-longformer')
('train_dataset', 'small.json')
('dev_dataset', 'small_3.json')
('batch_size', 2)
('gpus', '0')
('warmup', 1000)
('lr', 5e-05)
('val_every', 1.0)
('val_percent_check', 1.0)
('num_workers', 4)
('seed', 1234)
('epochs', 6)
('max_seq_len', 512)
('max_doc_len', 4096)
('max_num_answers', 64)
('max_question_len', 55)
('doc_stride', -1)
('ignore_seq_with_no_answers', False)
('disable_checkpointing', False)
('n_best_size', 20)
('max_answer_length', 30)
('regular_softmax_loss', False)
('test', False)
('model_path', None)
('no_progress_bar', False)
('attention_mode', 'sliding_chunks')
('fp32', False)
('train_percent', 1.0)
main


### Sandbox

In [ ]:
>>> from transformers import DistilBertTokenizer, DistilBertModel
>>> import torch
>>> tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
>>> model = DistilBertModel.from_pretrained('distilbert-base-uncased')
>>> inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
inputs